In [1]:

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from peerannot.models.aggregation.diagonal_multinomial_online import VectorizedDiagonalMultinomialOnlineMongo
from peerannot.models.aggregation.multinomial_binary_online import (
    VectorizedMultinomialBinaryOnlineMongo,
)

from peerannot.models.aggregation.types import ClassMapping, TaskMapping, WorkerMapping
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Batch 1: 4 tasks
batch1 = {
    "task_A": {
        "user_001": "Quercus robur",
        "user_002": "Betula pendula",
    },
    "task_B": {
        "user_003": "Pinus sylvestris",
    },
    "task_C": {
        "user_001": "Fagus sylvatica",
        "user_004": "Quercus robur",
    },
    "task_D": {
        "user_002": "Betula pendula",
        "user_005": "Acer platanoides",
    },
}



# Batch 2: 5 tasks
batch2 = {
    "task_A": {
        "user_003": "Pinus sylvestris",
        "user_004": "Quercus robur",
        "user_005": "Quercus robur",
    },
    "task_B": {
        "user_002": "Pinus sylvestris",
        "user_005": "Pinus sylvestris",
    },
    "task_E": {
        "user_001": "Fagus sylvatica",
        "user_002": "Fagus sylvatica",
    },
    "task_F": {
        "user_004": "Tilia cordata",
    },
    "task_G": {
        "user_003": "Pinus sylvestris",
        "user_001": "Acer platanoides",
        "user_005": "Fagus sylvatica",
    },
}





/home/jozef/Desktop/repos/peerannot/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# add more models (dropdown widget)
# one slider for 2 batches
# visual. module 
# visualize raw data
# add information about docker
# 

VectorizedMultinomialBinaryOnlineMongo is an online EM-based aggregation algorithm for crowdsourced labels, designed to work with sparse data stored in a MongoDB backend.
It extends SparseMongoOnlineAlgorithm.

It handles sparse task x worker x class matrices (`sparse.COO`) to store only non-zero label assignments.

Model supports online updates, meaning it can incrementally update worker reliabilities (pi) without recomputing everything from scratch.

VectorizedMultinomialBinaryOnlineMongo for each worker stores a single scalar -- current estimate of worker reliability.

In [3]:
from peerannot.models.aggregation.diagonal_multinomial_online import DiagonalMultinomialOnline, VectorizedDiagonalMultinomialOnlineMongo


model = VectorizedMultinomialBinaryOnlineMongo()
model.drop()

model.process_batch(batch1)
model.get_answers()




2025-10-14 15:18:36.731 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 000 | L=-1.034318 | delta=inf | 10.351s
2025-10-14 15:18:36.741 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 001 | L=-0.786676 | delta=2.39e-01 | 0.009s
2025-10-14 15:18:36.750 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 002 | L=-0.613312 | delta=2.20e-01 | 0.009s
2025-10-14 15:18:36.760 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 003 | L=-0.517546 | delta=1.56e-01 | 0.009s
2025-10-14 15:18:36.770 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 004 | L=-0.487210 | delta=5.86e-02 | 0.009s
2025-10-14 15:18:36.779 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 005 | L=-0.474244 | delta=2.66e-02 | 0.009s
2025-10-14 15:18:36.789 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 006 | L=-0.450693 | delta=4.97e-02 | 0.009s
2025-10-14 15:18:36.798 | DEBUG    | peerannot.helpers.logging:log

array(['Betula pendula', 'Pinus sylvestris', 'Betula pendula',
       'Betula pendula'], dtype='<U16')

1. Batch preparation:
`process_batch` calls methods  to prepare `batch_matrix`, a 3 dimensional **one-hot encoded** array of shape n_tasks x n_workers x n_classes and batch appropriate mappings, i.e.  mappings that keep track of position of tasks, workers and classes in the array.
In the above example the 3 dimensional array will be of shape 4 x 5 x 5 with following mapping.

```python
task_mapping={'task_A': 0, 'task_B': 1, 'task_C': 2, 'task_D': 3}
worker_mapping={'user_001': 0, 'user_002': 1, 'user_003': 2, 'user_004': 3, 'user_005': 4}
class_mapping={'Quercus robur': 0, 'Betula pendula': 1, 'Pinus sylvestris': 2, 'Fagus sylvatica': 3, 'Acer platanoides': 4}
```

2.Soft vote calculation:
The `batch_-_matrix` input array (n_tasks x n_workers x n_classes) is collapsed across workers to compute `batch_T`, a 2D array (n_tasks x n_classes) containing per-task class probability distributions.

Each row in batch_T is the normalized frequency of worker votes for that task, so it reflects soft voting (label proportions).


3. Iterative EM updates:
The prepared `batch_matrix` and `batch_T` are used in the iterative EM loop, that runs until convergence or maxiter iterations reached.
Inside each iteration:
1. M-step: Giben current posteriors (`batch_T`), estimate batch_rho (class prior distribution) and batch_pi (worker reliability)
2. E-step: Given new parameters (batch_pi, batch_rho) recomputer posteriors (`batch_T`)
3. log-likelihood calculation: log of total probability of observed labels under current parameters
4. convergence check


After the loop online updates are performed. Updattheed probabilities (`batch_T`, `batch_rho` and `batch_pi`) are pused into the store.


In [4]:

model.process_batch(batch2)

model.get_answers()
print(model.T.shape)


2025-10-14 15:18:37.011 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 000 | L=-0.644996 | delta=inf | 0.013s
2025-10-14 15:18:37.022 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 001 | L=-0.470418 | delta=2.71e-01 | 0.010s
2025-10-14 15:18:37.032 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 002 | L=-0.458180 | delta=2.60e-02 | 0.010s
2025-10-14 15:18:37.043 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 003 | L=-0.452681 | delta=1.20e-02 | 0.010s
2025-10-14 15:18:37.053 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 004 | L=-0.445831 | delta=1.51e-02 | 0.009s
2025-10-14 15:18:37.063 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 005 | L=-0.438633 | delta=1.61e-02 | 0.009s
2025-10-14 15:18:37.073 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 006 | L=-0.430680 | delta=1.81e-02 | 0.009s
2025-10-14 15:18:37.083 | DEBUG    | peerannot.helpers.logging:log_

(7, 6)


When we call above, the following happens:

1. Batch preparation
Just like with batch1, the new batch batch2 is transformed into a 3D one-hot encoded array `batch_matrix` (n_tasks x n_workers x n_classes). Mappings for tasks, workers, and classes are created or updated to reflect the current batch.

2.Soft vote calculation:
`batch_matrix` is collapsed along the worker dimension to compute `batch_T`, a 2D array (n_tasks x n_classes) representing the normalized frequency of worker votes per task. This gives the soft label distribution for the new tasks.

2a. Blending with previous estimates:
If previous task-class probabilities exist (from batch1), the new estimates are blended with the stored ones using a convex combination with weight gamma. This ensures continuity and prevents overwriting prior knowledge, allowing the model to update incrementally.


3. Iterative EM updates:
The EM loop runs on the new batch:

M-step: Given current posteriors (`batch_T`), estimate batch-specific parameters `batch_rho` (class priors) and `batch_pi` (worker reliabilities).

E-step: Recompute task posteriors (`batch_T`) using updated parameters.

Log-likelihood: Calculate the probability of observed labels under current parameters to monitor convergence.

Convergence check: Stop if the maximum number of iterations is reached or the change in log-likelihood is below a threshold.

Online update:
Updated probabilities (`batch_T`, `batch_rho`, `batch_pi`) are merged with the stored model state in the database, effectively making the model incremental and online.

Answer retrieval:
model.get_answers() returns the current task-class predictions, reflecting both the new batch and prior knowledge from batch1.

Following code cells demonstrate how the model works from the inside out.

We start by initializing the model and preparing mappings for tasks, workers, and classes. The batch is then transformed into a one-hot encoded matrix, which serves as input to the EM algorithm.

A custom EM loop (em_trace) is implemented to iteratively estimate:
- batch_rho: class priors
- batch_pi: worker reliabilities
- batch_T: posterior probabilities for each task

The loop logs intermediate states, including the log-likelihood, at each iteration. After convergence, an **online update** merges the batch estimates into the model.

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

import numpy as np
from typing import Dict, Any, Tuple


def get_global_params(model) -> Dict[str, Any]:
    """Safely fetch global parameters from model."""
    def safe_attr(attr):
        try:
            return getattr(model, attr)
        except Exception:
            return None

    return {
        "rho": safe_attr("rho"),
        "pi": safe_attr("pi"),
        "T": safe_attr("T"),
    }

def to_dense(mat):
    if hasattr(mat, "todense"):
        return mat.todense()
    return mat

def remap_param_to_global(local_param, local_map, global_coll):
    """
    Remap a local parameter (scalar, vector, or matrix per ID) to global order.

    Supports:
      - scalar: single numeric value
      - vector: one value per local index
      - matrix/tensor per ID (e.g., confusion matrices)
    """
    n = global_coll.count_documents({})

    # --- Handle scalar case early ---
    if np.isscalar(local_param) or np.ndim(local_param) == 0:
        return np.full((n,), float(local_param))

    # --- Determine shape of one entry ---
    if isinstance(local_param, dict):
        try:
            first_value = next(iter(local_param.values()))
        except StopIteration:
            first_value = 0.0
    else:
        # handle numpy arrays, lists, etc.
        first_value = local_param[0] if np.ndim(local_param) > 0 else local_param

    shape_suffix = np.shape(first_value)
    global_shape = (n,) + shape_suffix
    global_param = np.zeros(global_shape, dtype=float)

    # --- Fill according to mapping ---
    for k, v in local_map.items():
        doc = global_coll.find_one({"_id": k})
        if doc is None:
            continue

        if isinstance(local_param, dict):
            value = local_param.get(k, np.zeros(shape_suffix))
        else:
            if v >= len(local_param):
                continue
            value = local_param[v]

        global_param[doc["index"]] = to_dense(value)

    return global_param

def remap_vector_to_global(vec, local_map, global_coll):

    n = global_coll.count_documents({})
    global_vec = np.zeros(n)
    for k, v in local_map.items():
        doc = global_coll.find_one({"_id": k})
        if doc:
            print(f'{global_vec=}')
            print(f'{doc['index']=}')
            print(f'{to_dense(vec[v])=}')
            global_vec[doc["index"]] = to_dense(vec[v])
    return global_vec


def remap_T_to_global(batch_T, task_mapping, class_mapping, model):
    """Embed local T into global shape (task x class)."""
    n_tasks = model.task_mapping.count_documents({})
    n_classes = model.class_mapping.count_documents({})

    # Build local→global index maps
    task_map = {v: model.task_mapping.find_one({"_id": k})["index"] for k, v in task_mapping.items()}
    class_map = {v: model.class_mapping.find_one({"_id": k})["index"] for k, v in class_mapping.items()}

    # Initialize global T
    global_T = np.zeros((n_tasks, n_classes))
    
    for t_local, t_global in task_map.items():
        for c_local, c_global in class_map.items():
            global_T[t_global, c_global] = batch_T[t_local, c_local]
    
    return global_T





def em_trace(model, batch_matrix, task_mapping, worker_mapping, class_mapping,
             maxiter=50, epsilon=1e-6, prev_globals=None):
    """Runs EM for one batch and traces evolution."""
    traces = []
    prev_globals = prev_globals or {}

    batch_T = model._init_T(batch_matrix, task_mapping, class_mapping)
    
    i, eps, ll = 0, np.inf, []

    while i < maxiter and eps > epsilon:
        batch_rho, batch_pi = model._m_step(batch_matrix, batch_T)
        batch_T, batch_denom = model._e_step(batch_matrix, batch_pi, batch_rho)
        likeli = np.log(np.sum(batch_denom))
        ll.append(likeli)

        rho_global = remap_param_to_global(batch_rho, class_mapping, model.class_mapping)
        pi_global = remap_param_to_global(batch_pi, worker_mapping, model.worker_mapping)
        T_global = remap_T_to_global(batch_T, task_mapping, class_mapping, model)

        traces.append({
            "iter": i,
            "rho": rho_global.copy(),
            "pi": pi_global.copy(),
            "pi_tensor": model.build_batch_pi_tensor(pi_global.copy(), class_mapping, worker_mapping),
            "T": T_global.copy(), 
            "likelihood": likeli,
            **{f"global_{k}": v for k, v in prev_globals.items()},
        })
        if i > 0:
            eps = np.abs((ll[-1] - ll[-2]) / (np.abs(ll[-2]) + 1e-12))
        i += 1
    

    # Online update after convergence
    model._online_update(task_mapping, worker_mapping, class_mapping, batch_T, batch_rho, batch_pi)

    # attach final global snapshot
    globals_after = get_global_params(model)
    if traces:
        traces[-1].update({f"global_{k}": v for k, v in globals_after.items()})

    return traces, globals_after


def prepare_batch(model, batch) -> Tuple[Any, Dict[str, Any]]:
    """Prepares a single batch and returns (matrix, mappings)."""
    task_mapping, worker_mapping, class_mapping = {}, {}, {}
    model._prepare_mapping(batch, task_mapping, worker_mapping, class_mapping)
    
    # ensure indices exist in global mappings
    model.get_or_create_indices(model.task_mapping, list(task_mapping))
    model.get_or_create_indices(model.worker_mapping, list(worker_mapping))
    model.get_or_create_indices(model.class_mapping, list(class_mapping))

    batch_matrix = model._process_batch_to_matrix(batch, task_mapping, worker_mapping, class_mapping)
    mappings = {
        "task_mapping": task_mapping.copy(),
        "worker_mapping": worker_mapping.copy(),
        "class_mapping": class_mapping.copy(),
    }
    return batch_matrix, mappings


def run_em_for_batches(model, batches, maxiter=50):
    """Runs EM sequentially for multiple batches, carrying over global parameters."""
    model.drop()
    model.t = 1
    all_traces = []
    all_mappings = []
    prev_globals = {}

    for idx, batch in enumerate(batches, 1):
        batch_matrix, mappings = prepare_batch(model, batch)
        traces, globals_after = em_trace(
            model,
            batch_matrix,
            mappings["task_mapping"],
            mappings["worker_mapping"],
            mappings["class_mapping"],
            maxiter=maxiter,
            prev_globals=prev_globals,
        )
        for t in traces:
            t['batch_num'] = idx

        all_traces.extend(traces)
        all_mappings.append(mappings)
        prev_globals = globals_after

        print(f"Finished batch {idx} (iterations: {len(traces)})")

    return all_traces, all_mappings

class PlotlyEMVisualizer:
    """Interactive visualization for EM model states and worker confusion matrices."""

    def __init__(self, model, trace_list, batch_data_map, batch_mappings, pi_renderer=None):
        self.model = model
        self.trace_list = trace_list
        self.batch_data_map = batch_data_map
        self.batch_mappings = batch_mappings
        self.pi_renderer = pi_renderer or self._plot_pi

        self.labels = self._get_labels()
        self.n_workers = len(self.labels["workers"])
        self.fig = None
        self.frames = []

    # ---------- Public API ----------

    def show(self):
        """Build and display the interactive visualization."""
        self.fig = self._create_base_figure()
        self._build_frames()
        self._apply_layout()
        self.fig.update_layout(showlegend=False)
        self.fig.show()

    # ---------- Internal Helpers ----------

    def _plot_pi(self, pi, label_prefix="batch", worker_labels=None, class_labels=None):
        """Return one or more Plotly traces for pi depending on its shape."""
        if pi is None:
            return [go.Bar(y=[0], x=["none"], name=f"{label_prefix}_pi")]

        pi = np.array(pi)

        # scalar pi — just one bar
        if pi.ndim == 0:
            return [go.Bar(y=[pi.item()], x=[f"{label_prefix}_pi"], name=f"{label_prefix}_pi")]

        # vector pi — reliability per worker
        if pi.ndim == 1:
            return [go.Bar(y=pi, x=worker_labels or list(range(len(pi))), name=f"{label_prefix}_pi")]

        # matrix pi — worker x class reliability
        if pi.ndim == 2:
            return [go.Heatmap(
                z=pi,
                x=class_labels or list(range(pi.shape[1])),
                y=worker_labels or list(range(pi.shape[0])),
                colorscale="Viridis",
                showscale=False,
                name=f"{label_prefix}_pi",
            )]

        # tensor pi — full confusion matrix per worker
        if pi.ndim == 3:
            traces = []
            for j, worker in enumerate(worker_labels or range(pi.shape[0])):
                traces.append(
                    go.Heatmap(
                        z=pi[j],
                        x=class_labels or list(range(pi.shape[2])),
                        y=class_labels or list(range(pi.shape[1])),
                        colorscale="Viridis",
                        showscale=False,
                        name=f"{label_prefix}_pi[{worker}]",
                    )
                )
            return traces

        raise ValueError(f"Unsupported pi dimensionality: {pi.ndim}")

    def _get_labels(self):
        """Extract ordered task, worker, and class labels from model mappings."""
        get_ids = lambda mapping: [doc["_id"] for doc in mapping.find().sort("index", 1)]
        return {
            "tasks": get_ids(self.model.task_mapping),
            "workers": get_ids(self.model.worker_mapping),
            "classes": get_ids(self.model.class_mapping),
        }

    def _create_base_figure(self):
        """Create a multi-subplot Plotly figure with placeholders."""
        task_labels = self.labels["tasks"]
        worker_labels = self.labels["workers"]
        class_labels = self.labels["classes"]
        total_cols = max(3, self.n_workers)

        fig = make_subplots(
            rows=3, cols=total_cols,
            row_heights=[0.25, 0.25, 0.4],
            vertical_spacing=0.12,
            subplot_titles=[
                "batch_rho (class priors)", "batch_T (local task x class)", "batch_pi (worker reliability)",
                "user votes", "global_rho (class priors)", "global_T (local task x class)", "global_pi (worker reliability)"
            ] + [f"batch_pi of {w}" for w in worker_labels] + ["Batch Table"],
            specs=[
                [{"type": "bar"}, {"type": "heatmap"}, {"type": "bar"}, {"type": "table", "rowspan": 2}]
                + [None] * (total_cols - 4),
                [{"type": "bar"}, {"type": "heatmap"}, {"type": "bar"}, None]
                + [None] * (total_cols - 4),
                [{"type": "heatmap"} for _ in range(self.n_workers)] + [None] * (total_cols - self.n_workers)
            ],
        )

        # --- Placeholders for EM matrices ---
        fig.add_trace(go.Bar(y=[0], x=class_labels), row=1, col=1)
        fig.add_trace(go.Heatmap(z=[[0]], x=class_labels, y=task_labels), row=1, col=2)
        fig.add_trace(go.Bar(y=[0], x=worker_labels), row=1, col=3)
        fig.add_trace(go.Bar(y=[0], x=class_labels), row=2, col=1)
        fig.add_trace(go.Heatmap(z=[[0]], x=class_labels, y=task_labels), row=2, col=2)
        fig.add_trace(go.Bar(y=[0], x=worker_labels), row=2, col=3)

        # --- Worker confusion matrix placeholders ---
        for i in range(self.n_workers):
            fig.add_trace(
                go.Heatmap(
                    z=np.zeros((len(class_labels), len(class_labels))),
                    x=class_labels,
                    y=class_labels,
                    colorscale="Viridis",
                    showscale=False,
                ),
                row=3, col=i + 1,
            )

        # --- Empty table placeholder ---
        fig.add_trace(
            go.Table(
                header=dict(values=["Task", "User", "Species"], fill_color="lightgrey", align="left"),
                cells=dict(values=[[], [], []]),
            ),
            row=1, col=4,
        )

        return fig

    def _build_frames(self):
        """Generate all animation frames."""
        for i, state in enumerate(self.trace_list):
            batch_num = state.get("batch_num", "?")
            batch_data = self.batch_data_map.get(batch_num, {})
            batch_mapping = next(
                (m for idx, m in enumerate(self.batch_mappings) if idx + 1 == batch_num), None
            )
            batch_class_labels = (
                list(batch_mapping["class_mapping"].keys())
                if batch_mapping
                else self.labels["classes"]
            )

            frame_data = self._make_frame_data(state, batch_class_labels, batch_data)

            frame = go.Frame(
                data=frame_data,
                name=f"iter_{i}",
                layout=go.Layout(
                    title_text=(
                        f"Batch {batch_num}, Iteration {state['iter']} - "
                        f"LL: {state.get('likelihood', 0.0):.3f}"
                    )
                ),
            )
            self.frames.append(frame)

        self.fig.frames = self.frames

    def _make_frame_data(self, state, batch_class_labels, batch_data):
        """Create all traces for one animation frame."""
        to_dense = lambda x: np.array(x) if x is not None else None

        task_labels = self.labels["tasks"]
        worker_labels = self.labels["workers"]
        all_class_labels = self.labels["classes"]

        # Extract matrices
        batch_rho = to_dense(state.get("rho"))
        batch_T = to_dense(state.get("T"))
        batch_pi = to_dense(state.get("pi"))
        global_rho = to_dense(state.get("global_rho"))
        global_T = to_dense(state.get("global_T"))
        global_pi = to_dense(state.get("global_pi"))
        pi_tensor = to_dense(state.get("pi_tensor"))

        # Build user table
        rows = [
            (task, user, species)
            for task, users in (batch_data or {}).items()
            for user, species in users.items()
        ]
        cols = list(zip(*rows)) if rows else [[], [], []]


        # pi plotting
        batch_pi_traces = self._plot_pi(
        state.get("pi"),
        label_prefix="batch",
        worker_labels=self.labels["workers"],
        class_labels=self.labels["classes"]
    )
        global_pi_traces = self._plot_pi(
        state.get("global_pi"),
        label_prefix="global",
        worker_labels=self.labels["workers"],
        class_labels=self.labels["classes"]
    )

        # EM components
        frame_data = [
            go.Bar(y=batch_rho.flatten() if batch_rho is not None else [0], x=batch_class_labels),
            go.Heatmap(z=batch_T if batch_T is not None else np.zeros((1, 1)), x=batch_class_labels, y=task_labels, showscale=False),
        ] + batch_pi_traces + [
            go.Bar(y=global_rho.flatten() if global_rho is not None else [0], x=all_class_labels),
            go.Heatmap(z=global_T if global_T is not None else np.zeros((1, 1)), x=all_class_labels, y=task_labels, showscale=False),
        ] + global_pi_traces

        # Worker confusion matrices
        if pi_tensor is not None:
            for j in range(self.n_workers):
                worker_pi = pi_tensor[j] if pi_tensor.ndim == 3 else pi_tensor
                frame_data.append(
                    go.Heatmap(
                        z=worker_pi,
                        x=batch_class_labels,
                        y=batch_class_labels,
                        colorscale="Viridis",
                        showscale=False,
                    )
                )
        else:
            for _ in range(self.n_workers):
                frame_data.append(
                    go.Heatmap(
                        z=np.zeros((len(batch_class_labels), len(batch_class_labels))),
                        x=batch_class_labels,
                        y=batch_class_labels,
                        colorscale="Viridis",
                        showscale=False,
                    )
                )

        # Append user voting table
        frame_data.append(self._make_table_trace(cols))
        return frame_data

    def _make_table_trace(self, cols):
        """Return a table trace for the given columns."""
        return go.Table(
            header=dict(values=["Task", "User", "Species"], fill_color="lightgrey", align="left"),
            cells=dict(values=cols, align="left", fill_color=[["white"] * len(cols[0])]),
        )

    def _apply_layout(self):
        """Apply layout, buttons, and slider configuration."""
        self.fig.update_layout(
            title=f"Iteration 0 - Log-likelihood: {self.trace_list[0].get('likelihood', 0.0):.3f}",
            height=1200,
            width=320 * self.n_workers,
            template="plotly_white",

            sliders=[{
                "x": 0.1, "y": -0.1,
                "steps": [
                    {"args": [[f"iter_{i}"], {"frame": {"duration": 0, "redraw": True}}],
                     "label": str(i), "method": "animate"}
                    for i in range(len(self.frames))
                ],
            }],
        )


# After running your EM:
# model = VectorizedDiagonalMultinomialOnlineMongo()
model = VectorizedMultinomialBinaryOnlineMongo()
combined_traces, batch_mappings = run_em_for_batches(model, [batch1, batch2], maxiter=50)
batch_data_map = {1: batch1, 2: batch2}

viz = PlotlyEMVisualizer(
    model=model,
    trace_list=combined_traces,
    batch_data_map=batch_data_map,
    batch_mappings=batch_mappings,
)
viz.show()


2025-10-14 15:18:37.899 | DEBUG    | peerannot.helpers.logging:mongo_timer:16 - [Mongo] online update class probs took 0.013s


Finished batch 1 (iterations: 16)


2025-10-14 15:18:38.531 | DEBUG    | peerannot.helpers.logging:mongo_timer:16 - [Mongo] online update class probs took 0.002s


Finished batch 2 (iterations: 20)


In [6]:
from peerannot.models.aggregation.diagonal_multinomial_online import VectorizedDiagonalMultinomialOnlineMongo
from peerannot.models.aggregation.flat_single_binomial_online import VectorizedFlatSingleBinomialOnlineMongo
from peerannot.models.aggregation.multinomial_binary_online import VectorizedMultinomialBinaryOnlineMongo
from peerannot.models.aggregation.poled_flat_diagonal_online import VectorizedPooledFlatDiagonalOnlineMongo
from peerannot.models.aggregation.pooled_diagonal_multinomial_online import VectorizedPooledDiagonalMultinomialOnlineMongo
from peerannot.models.aggregation.pooled_flat_single_binomial_online import VectorizedPooledFlatSingleBinomialOnlineMongo
from peerannot.models.aggregation.pooled_multinomial_binary_online import (
    VectorizedPooledMultinomialBinaryOnlineMongo,
)
from peerannot.models.aggregation.pooled_multinomial_online import VectorizedPooledMultinomialOnlineMongo

# model = VectorizedFlatSingleBinomialOnlineMongo() # nie działa

model = VectorizedFlatSingleBinomialOnlineMongo()
combined_traces, batch_mappings = run_em_for_batches(model, [batch1, batch2], maxiter=50)
batch_data_map = {1: batch1, 2: batch2}

viz = PlotlyEMVisualizer(
    model=model,
    trace_list=combined_traces,
    batch_data_map=batch_data_map,
    batch_mappings=batch_mappings,
)
viz.show()


T.sum(axis=1, keepdims=True)=<COO: shape=(4, 1), dtype=float64, nnz=4, fill_value=0.0>
       0
  ┌         ┐
0 │ 0.05371 │
1 │  0.25   │
2 │ 0.02757 │
3 │ 0.02757 │
  └         ┘
T.sum(axis=1, keepdims=True)=<COO: shape=(4, 1), dtype=float64, nnz=4, fill_value=0.0>
       0
  ┌         ┐
0 │ 0.05088 │
1 │ 0.3453  │
2 │ 0.02522 │
3 │ 0.02562 │
  └         ┘
T.sum(axis=1, keepdims=True)=<COO: shape=(4, 1), dtype=float64, nnz=4, fill_value=0.0>
       0
  ┌         ┐
0 │ 0.04973 │
1 │  0.479  │
2 │ 0.02304 │
3 │ 0.02296 │
  └         ┘
T.sum(axis=1, keepdims=True)=<COO: shape=(4, 1), dtype=float64, nnz=4, fill_value=0.0>
       0
  ┌         ┐
0 │  0.067  │
1 │ 0.6992  │
2 │ 0.02804 │
3 │ 0.02728 │
  └         ┘
T.sum(axis=1, keepdims=True)=<COO: shape=(4, 1), dtype=float64, nnz=4, fill_value=0.0>
       0
  ┌         ┐
0 │ 0.1154  │
1 │ 0.9508  │
2 │ 0.04575 │
3 │ 0.0435  │
  └         ┘
T.sum(axis=1, keepdims=True)=<COO: shape=(4, 1), dtype=float64, nnz=4, fill_value=0.0>
       0
  ┌ 

/home/jozef/Desktop/repos/peerannot/.venv/lib/python3.12/site-packages/sparse/_umath.py:608: RuntimeWarning:

divide by zero encountered in divide

/home/jozef/Desktop/repos/peerannot/.venv/lib/python3.12/site-packages/sparse/_umath.py:608: RuntimeWarning:

invalid value encountered in divide

/home/jozef/Desktop/repos/peerannot/.venv/lib/python3.12/site-packages/sparse/_umath.py:519: RuntimeWarning:

invalid value encountered in multiply



AttributeError: 'numpy.ndarray' object has no attribute 'todense'

In [7]:
model = VectorizedFlatSingleBinomialOnlineMongo()
model.drop()
model.process_batch(batch1)

2025-10-14 15:18:43.050 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 000 | L=-1.024869 | delta=inf | 0.014s
2025-10-14 15:18:43.064 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 001 | L=-0.805144 | delta=2.14e-01 | 0.013s
2025-10-14 15:18:43.080 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 002 | L=-0.553808 | delta=3.12e-01 | 0.015s
2025-10-14 15:18:43.096 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 003 | L=-0.196556 | delta=6.45e-01 | 0.015s
2025-10-14 15:18:43.110 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 004 | L=0.144482 | delta=1.74e+00 | 0.013s
2025-10-14 15:18:43.125 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 005 | L=0.201247 | delta=3.93e-01 | 0.015s
2025-10-14 15:18:43.143 | DEBUG    | peerannot.helpers.logging:log_em_iter:26 - [EM] Iter 006 | L=0.200028 | delta=6.06e-03 | 0.017s


T.sum(axis=1, keepdims=True)=<COO: shape=(4, 1), dtype=float64, nnz=4, fill_value=0.0>
       0
  ┌         ┐
0 │ 0.05371 │
1 │  0.25   │
2 │ 0.02757 │
3 │ 0.02757 │
  └         ┘
T.sum(axis=1, keepdims=True)=<COO: shape=(4, 1), dtype=float64, nnz=4, fill_value=0.0>
       0
  ┌         ┐
0 │ 0.05088 │
1 │ 0.3453  │
2 │ 0.02522 │
3 │ 0.02562 │
  └         ┘
T.sum(axis=1, keepdims=True)=<COO: shape=(4, 1), dtype=float64, nnz=4, fill_value=0.0>
       0
  ┌         ┐
0 │ 0.04973 │
1 │  0.479  │
2 │ 0.02304 │
3 │ 0.02296 │
  └         ┘
T.sum(axis=1, keepdims=True)=<COO: shape=(4, 1), dtype=float64, nnz=4, fill_value=0.0>
       0
  ┌         ┐
0 │  0.067  │
1 │ 0.6992  │
2 │ 0.02804 │
3 │ 0.02728 │
  └         ┘
T.sum(axis=1, keepdims=True)=<COO: shape=(4, 1), dtype=float64, nnz=4, fill_value=0.0>
       0
  ┌         ┐
0 │ 0.1154  │
1 │ 0.9508  │
2 │ 0.04575 │
3 │ 0.0435  │
  └         ┘
T.sum(axis=1, keepdims=True)=<COO: shape=(4, 1), dtype=float64, nnz=4, fill_value=0.0>
       0
  ┌ 

/home/jozef/Desktop/repos/peerannot/.venv/lib/python3.12/site-packages/sparse/_umath.py:608: RuntimeWarning:

divide by zero encountered in divide

/home/jozef/Desktop/repos/peerannot/.venv/lib/python3.12/site-packages/sparse/_umath.py:608: RuntimeWarning:

invalid value encountered in divide

/home/jozef/Desktop/repos/peerannot/.venv/lib/python3.12/site-packages/sparse/_umath.py:519: RuntimeWarning:

invalid value encountered in multiply



AttributeError: 'numpy.ndarray' object has no attribute 'todense'